Part - 2:
=========

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import DataLoader, TensorDataset
import time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, roc_curve, auc

In [2]:
df = pd.read_csv('dataset.csv')
print(df.describe())

               f3      target
count  766.000000  766.000000
mean    69.118799    0.349869
std     19.376901    0.477240
min      0.000000    0.000000
25%     62.500000    0.000000
50%     72.000000    0.000000
75%     80.000000    1.000000
max    122.000000    1.000000


In [3]:
# we see that there are alphabets in numeric columns so we replace them with mean of that respective column
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col].fillna(df[col].mean(), inplace=True)
df.tail

<bound method NDFrame.tail of             f1     f2  f3         f4          f5        f6        f7  target
0     6.000000  148.0  72  35.000000    0.000000  33.60000  0.627000       1
1     1.000000   85.0  66  29.000000    0.000000  26.60000  0.351000       0
2     8.000000  183.0  64   0.000000    0.000000  23.30000  0.672000       1
3     1.000000   89.0  66  23.000000   94.000000  28.10000  0.167000       0
4     0.000000  137.0  40  35.000000  168.000000  43.10000  2.288000       1
..         ...    ...  ..        ...         ...       ...       ...     ...
761   9.000000   89.0  62   0.000000    0.000000  22.50000  0.472128       0
762  10.000000  101.0  76  48.000000  180.000000  31.99817  0.171000       0
763   2.000000  122.0  70  27.000000   80.091503  36.80000  0.340000       0
764   3.849673  121.0  72  23.000000  112.000000  26.20000  0.245000       0
765   1.000000  126.0  60  20.542484    0.000000  30.10000  0.349000       1

[766 rows x 8 columns]>

In [4]:
scaler = StandardScaler()
df1 = df.drop('target', axis=1)
column = df1.columns
df1[column] = scaler.fit_transform(df1[column])
df1['target'] = df['target']
df1.tail

<bound method NDFrame.tail of                f1        f2        f3            f4        f5            f6  \
0    6.382139e-01  0.849057  0.148790  9.070150e-01 -0.695096  2.030729e-01   
1   -8.457789e-01 -1.125480 -0.161060  5.305956e-01 -0.695096 -6.843559e-01   
2    1.231811e+00  1.946022 -0.264343 -1.288765e+00 -0.695096 -1.102715e+00   
3   -8.457789e-01 -1.000113 -0.161060  1.541762e-01  0.120709 -4.941926e-01   
4   -1.142577e+00  0.504297 -1.503740  9.070150e-01  0.762938  1.407440e+00   
..            ...       ...       ...           ...       ...           ...   
761  1.528610e+00 -1.000113 -0.367626 -1.288765e+00 -0.695096 -1.204136e+00   
762  1.825408e+00 -0.624010  0.355356  1.722590e+00  0.867083  4.503972e-16   
763 -5.489804e-01  0.034169  0.045507  4.051224e-01  0.000000  6.087546e-01   
764  1.318050e-16  0.002827  0.148790  1.541762e-01  0.276927 -7.350661e-01   
765 -8.457789e-01  0.159536 -0.470909  2.228851e-16 -0.695096 -2.406415e-01   

               f7  ta

In [5]:
X = df.drop('target', axis=1)  # Replace 'target_column' with the actual target column name
Y = df['target']
ros = RandomOverSampler(random_state=42)
X, Y = ros.fit_resample(X, Y)
# Split the data into training and testing sets (80% train, 20% test by default)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.175, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
Y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
print("X_train size: ", X_train_tensor.shape)
print("Y_train size: ", Y_train_tensor.shape)
print("X_val size: ", X_val_tensor.shape)
print("Y_val size: ", Y_val_tensor.shape)
print("X_test size: ", X_test_tensor.shape)
print("Y_test size: ", Y_test_tensor.shape)

X_train size:  torch.Size([697, 7])
Y_train size:  torch.Size([697])
X_val size:  torch.Size([149, 7])
Y_val size:  torch.Size([149])
X_test size:  torch.Size([150, 7])
Y_test size:  torch.Size([150])


Tuning hyperparameters:
=======================
1. Dropout value:
-----------------

In [6]:
class NN_dropout(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(NN_dropout, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [7]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
dropout_rate = [0.3, 0.5, 0.7]
for dropout in dropout_rate:
    model = NN_dropout(input_size, hidden_size, output_size, dropout)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.007)

    num_epochs = 300
    batch_size = 64
    best_val_loss = float('inf')

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            predictions = torch.round(torch.sigmoid(outputs))
            correct_train += (predictions.squeeze(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
    
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

                val_predictions = torch.round(torch.sigmoid(val_outputs))
                correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = correct_val / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if (epoch+1)%10==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
    # Evaluation on the test data
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    test_losses = [test_loss] * num_epochs
    test_accuracies = [test_accuracy] * num_epochs

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Dropout: {dropout}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.6610, Validation Loss: 0.6292, Training Accuracy: 0.6327, Validation Accuracy: 0.6779
Epoch 20/300, Training Loss: 0.6358, Validation Loss: 0.6031, Training Accuracy: 0.6643, Validation Accuracy: 0.7047
Epoch 30/300, Training Loss: 0.6222, Validation Loss: 0.5934, Training Accuracy: 0.6772, Validation Accuracy: 0.7181
Epoch 40/300, Training Loss: 0.6214, Validation Loss: 0.5971, Training Accuracy: 0.6786, Validation Accuracy: 0.7181
Epoch 50/300, Training Loss: 0.6338, Validation Loss: 0.6155, Training Accuracy: 0.6729, Validation Accuracy: 0.6309
Epoch 60/300, Training Loss: 0.6109, Validation Loss: 0.5962, Training Accuracy: 0.6829, Validation Accuracy: 0.6644
Epoch 70/300, Training Loss: 0.6008, Validation Loss: 0.5880, Training Accuracy: 0.6915, Validation Accuracy: 0.6711
Epoch 80/300, Training Loss: 0.6279, Validation Loss: 0.5877, Training Accuracy: 0.6614, Validation Accuracy: 0.7114
Epoch 90/300, Training Loss: 0.6075, Validation Loss: 0.6006, Tr


2. Batch size:
---------------

In [8]:
class NN_batchsize_lr(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN_batchsize_lr, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [9]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.007)
batch_size_list = [32, 64, 128]
for batch in batch_size_list:
    
    num_epochs = 300
    batch_size = batch
    best_val_loss = float('inf')

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            predictions = torch.round(torch.sigmoid(outputs))
            correct_train += (predictions.squeeze(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
    
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

                val_predictions = torch.round(torch.sigmoid(val_outputs))
                correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = correct_val / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if (epoch+1)%10==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
    # Evaluation on the test data
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    test_losses = [test_loss] * num_epochs
    test_accuracies = [test_accuracy] * num_epochs

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Batch size: {batch_size}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.6837, Validation Loss: 0.6709, Training Accuracy: 0.5825, Validation Accuracy: 0.6107
Epoch 20/300, Training Loss: 0.6645, Validation Loss: 0.6479, Training Accuracy: 0.5983, Validation Accuracy: 0.6040
Epoch 30/300, Training Loss: 0.6622, Validation Loss: 0.6512, Training Accuracy: 0.6055, Validation Accuracy: 0.6040
Epoch 40/300, Training Loss: 0.6578, Validation Loss: 0.6553, Training Accuracy: 0.6313, Validation Accuracy: 0.5906
Epoch 50/300, Training Loss: 0.6414, Validation Loss: 0.6252, Training Accuracy: 0.6284, Validation Accuracy: 0.6443
Epoch 60/300, Training Loss: 0.6524, Validation Loss: 0.6383, Training Accuracy: 0.6255, Validation Accuracy: 0.6040
Epoch 70/300, Training Loss: 0.6464, Validation Loss: 0.6493, Training Accuracy: 0.6298, Validation Accuracy: 0.5772
Epoch 80/300, Training Loss: 0.6356, Validation Loss: 0.6166, Training Accuracy: 0.6643, Validation Accuracy: 0.7181
Epoch 90/300, Training Loss: 0.6172, Validation Loss: 0.5776, Tr

3. Learning Rate:
-------------------

In [10]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning_rates = [0.001, 0.007, 0.01]
for learning in learning_rates:
    optimizer = optim.Adam(model.parameters(), lr=learning)
    num_epochs = 300
    batch_size = 64
    best_val_loss = float('inf')

    train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            predictions = torch.round(torch.sigmoid(outputs))
            correct_train += (predictions.squeeze(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
    
        model.eval()
        val_loss = 0.0
        correct_val = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

                val_predictions = torch.round(torch.sigmoid(val_outputs))
                correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_accuracy = correct_val / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if (epoch+1)%10==0:
            print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
    # Evaluation on the test data
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    test_losses = [test_loss] * num_epochs
    test_accuracies = [test_accuracy] * num_epochs

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    print(f"Learning Rate: {learning}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.7759, Validation Loss: 0.6789, Training Accuracy: 0.5065, Validation Accuracy: 0.6443
Epoch 20/300, Training Loss: 0.7066, Validation Loss: 0.6834, Training Accuracy: 0.5308, Validation Accuracy: 0.5973
Epoch 30/300, Training Loss: 0.6874, Validation Loss: 0.6841, Training Accuracy: 0.5509, Validation Accuracy: 0.5503
Epoch 40/300, Training Loss: 0.6799, Validation Loss: 0.6791, Training Accuracy: 0.5567, Validation Accuracy: 0.5906
Epoch 50/300, Training Loss: 0.6739, Validation Loss: 0.6731, Training Accuracy: 0.5954, Validation Accuracy: 0.5906
Epoch 60/300, Training Loss: 0.6665, Validation Loss: 0.6667, Training Accuracy: 0.6011, Validation Accuracy: 0.5772
Epoch 70/300, Training Loss: 0.6602, Validation Loss: 0.6631, Training Accuracy: 0.6098, Validation Accuracy: 0.5973
Epoch 80/300, Training Loss: 0.6588, Validation Loss: 0.6617, Training Accuracy: 0.6069, Validation Accuracy: 0.5906
Epoch 90/300, Training Loss: 0.6651, Validation Loss: 0.6540, Tr

#### Base Model:

In [24]:
class NN_batchsize_lr(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN_batchsize_lr, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [29]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.007)
    
num_epochs = 300
batch_size = 64
best_val_loss = float('inf')

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
test_accuracies_base_model = []
start_time_base_model = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predictions = torch.round(torch.sigmoid(outputs))
        correct_train += (predictions.squeeze(1) == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    

    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

            val_predictions = torch.round(torch.sigmoid(val_outputs))
            correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    if (epoch+1)%10==0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
            f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    test_accuracies_base_model.append(test_accuracy)  # Store test accuracy for this epoch

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
end_time_base_model = time.time()
print(f"Training completed in: {(end_time_base_model - start_time_base_model) / 60:.2f} minutes")
        
# Evaluation on the test data
model.eval()
y_true = []
y_pred = []
test_loss = 0.0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
        test_loss += loss.item() * test_inputs.size(0)
        predictions = torch.round(torch.sigmoid(test_outputs))
        y_true.extend(test_labels.tolist())
        y_pred.extend(predictions.squeeze(1).tolist())

test_loss /= len(test_loader.dataset)
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

test_losses = [test_loss] * num_epochs
test_accuracies = [test_accuracy] * num_epochs

y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Batch size: {batch_size}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")



Epoch 10/300, Training Loss: 0.6521, Validation Loss: 0.6205, Training Accuracy: 0.6528, Validation Accuracy: 0.7248
Epoch 10/300, Test Loss: 0.6444, Test Accuracy: 0.6200
Epoch 20/300, Training Loss: 0.6543, Validation Loss: 0.6081, Training Accuracy: 0.6356, Validation Accuracy: 0.7114
Epoch 20/300, Test Loss: 0.6368, Test Accuracy: 0.6267
Epoch 30/300, Training Loss: 0.6155, Validation Loss: 0.5712, Training Accuracy: 0.6901, Validation Accuracy: 0.7181
Epoch 30/300, Test Loss: 0.6047, Test Accuracy: 0.6533
Epoch 40/300, Training Loss: 0.6216, Validation Loss: 0.5872, Training Accuracy: 0.6743, Validation Accuracy: 0.6846
Epoch 40/300, Test Loss: 0.6132, Test Accuracy: 0.6400
Epoch 50/300, Training Loss: 0.6011, Validation Loss: 0.5903, Training Accuracy: 0.6944, Validation Accuracy: 0.6711
Epoch 50/300, Test Loss: 0.6128, Test Accuracy: 0.6600
Epoch 60/300, Training Loss: 0.6024, Validation Loss: 0.5821, Training Accuracy: 0.6901, Validation Accuracy: 0.6913
Epoch 60/300, Test Loss

#### Early Stopping:

In [12]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning = 0.007
optimizer = optim.Adam(model.parameters(), lr=learning)
num_epochs = 300
batch_size = 64
best_val_loss = float('inf')  # Initialize with a very large value
early_stopping_patience = 10  # Number of epochs to wait for improvement before stopping
no_improvement_epochs = 0  # Counter for epochs without improvement

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predictions = torch.round(torch.sigmoid(outputs))
        correct_train += (predictions.squeeze(1) == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

            val_predictions = torch.round(torch.sigmoid(val_outputs))
            correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset)
    
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    # Early Stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_epochs = 0  # Reset counter if validation loss improves
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        no_improvement_epochs += 1  # Increase counter if no improvement
    
    if no_improvement_epochs >= early_stopping_patience:
        print(f"Early stopping triggered after {epoch+1} epochs!")
        break

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
              f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Load the best model saved during training
model.load_state_dict(torch.load('best_model.pth'))

# Evaluation on the test data
model.eval()
y_true = []
y_pred = []
test_loss = 0.0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
        test_loss += loss.item() * test_inputs.size(0)
        predictions = torch.round(torch.sigmoid(test_outputs))
        y_true.extend(test_labels.tolist())
        y_pred.extend(predictions.squeeze(1).tolist())

test_loss /= len(test_loader.dataset)
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

test_losses = [test_loss] * num_epochs
test_accuracies = [test_accuracy] * num_epochs

y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Learning Rate: {learning}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Epoch 10/300, Training Loss: 0.6845, Validation Loss: 0.6709, Training Accuracy: 0.5653, Validation Accuracy: 0.5906
Epoch 20/300, Training Loss: 0.6787, Validation Loss: 0.6732, Training Accuracy: 0.5983, Validation Accuracy: 0.5302
Epoch 30/300, Training Loss: 0.6505, Validation Loss: 0.6575, Training Accuracy: 0.6255, Validation Accuracy: 0.6107
Epoch 40/300, Training Loss: 0.6574, Validation Loss: 0.6562, Training Accuracy: 0.6026, Validation Accuracy: 0.5906
Epoch 50/300, Training Loss: 0.6579, Validation Loss: 0.6473, Training Accuracy: 0.6298, Validation Accuracy: 0.6376
Early stopping triggered after 57 epochs!
Test Loss: 0.6300, Test Accuracy: 0.6733
Learning Rate: 0.007
Test Accuracy: 0.6733
Test Precision: 0.6774
Test Recall: 0.5915
Test F1 Score: 0.6316


#### Learning Rate Scheduler

In [13]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning = 0.007
optimizer = optim.Adam(model.parameters(), lr=learning)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

num_epochs = 300
batch_size = 64
best_val_loss = float('inf')

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predictions = torch.round(torch.sigmoid(outputs))
        correct_train += (predictions.squeeze(1) == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

            val_predictions = torch.round(torch.sigmoid(val_outputs))
            correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    # Learning rate scheduler step
    scheduler.step(val_loss)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
              f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Evaluation on the test data
model.eval()
y_true = []
y_pred = []
test_loss = 0.0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
        test_loss += loss.item() * test_inputs.size(0)
        predictions = torch.round(torch.sigmoid(test_outputs))
        y_true.extend(test_labels.tolist())
        y_pred.extend(predictions.squeeze(1).tolist())

test_loss /= len(test_loader.dataset)
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

test_losses = [test_loss] * num_epochs
test_accuracies = [test_accuracy] * num_epochs

y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Epoch 10/300, Training Loss: 0.6831, Validation Loss: 0.6766, Training Accuracy: 0.5983, Validation Accuracy: 0.5973
Epoch 20/300, Training Loss: 0.6781, Validation Loss: 0.6690, Training Accuracy: 0.5854, Validation Accuracy: 0.5839
Epoch 00025: reducing learning rate of group 0 to 7.0000e-04.
Epoch 30/300, Training Loss: 0.6573, Validation Loss: 0.6515, Training Accuracy: 0.6298, Validation Accuracy: 0.6376
Epoch 40/300, Training Loss: 0.6563, Validation Loss: 0.6473, Training Accuracy: 0.6428, Validation Accuracy: 0.6510
Epoch 00045: reducing learning rate of group 0 to 7.0000e-05.
Epoch 50/300, Training Loss: 0.6521, Validation Loss: 0.6458, Training Accuracy: 0.6456, Validation Accuracy: 0.6510
Epoch 00051: reducing learning rate of group 0 to 7.0000e-06.
Epoch 00057: reducing learning rate of group 0 to 7.0000e-07.
Epoch 60/300, Training Loss: 0.6490, Validation Loss: 0.6457, Training Accuracy: 0.6341, Validation Accuracy: 0.6577
Epoch 00063: reducing learning rate of group 0 to 

#### Batch Normalization

In [26]:
class NN_batchnorm(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN_batchnorm, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Add BatchNorm
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(self.bn1(x))  # Apply BatchNorm
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(self.bn2(x))  # Apply BatchNorm
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(self.bn3(x))  # Apply BatchNorm
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Replace the model with the one using batch normalization
model = NN_batchnorm(input_size, hidden_size, output_size)


In [30]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchnorm(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning = 0.007
optimizer = optim.Adam(model.parameters(), lr=learning)
num_epochs = 300
batch_size = 64
best_val_loss = float('inf')

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
test_accuracies_improved = []
start_time_improved = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predictions = torch.round(torch.sigmoid(outputs))
        correct_train += (predictions.squeeze(1) == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    

    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

            val_predictions = torch.round(torch.sigmoid(val_outputs))
            correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    if (epoch+1)%10==0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
            f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_accuracy = accuracy_score(y_true, y_pred)
    test_accuracies_improved.append(test_accuracy)  # Store test accuracy for this epoch

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

end_time_improved = time.time()
print(f"Training completed in: {(end_time_improved - start_time_improved) / 60:.2f} minutes")
# Evaluation on the test data
model.eval()
y_true = []
y_pred = []
test_loss = 0.0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
        test_loss += loss.item() * test_inputs.size(0)
        predictions = torch.round(torch.sigmoid(test_outputs))
        y_true.extend(test_labels.tolist())
        y_pred.extend(predictions.squeeze(1).tolist())

test_loss /= len(test_loader.dataset)
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

test_losses = [test_loss] * num_epochs
test_accuracy = [test_accuracy] * num_epochs

y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Learning Rate: {learning}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

Epoch 10/300, Training Loss: 0.5063, Validation Loss: 0.5475, Training Accuracy: 0.7561, Validation Accuracy: 0.7047
Epoch 10/300, Test Loss: 0.5215, Test Accuracy: 0.7267
Epoch 20/300, Training Loss: 0.4938, Validation Loss: 0.5697, Training Accuracy: 0.7547, Validation Accuracy: 0.6711
Epoch 20/300, Test Loss: 0.5292, Test Accuracy: 0.6933
Epoch 30/300, Training Loss: 0.4908, Validation Loss: 0.5579, Training Accuracy: 0.7618, Validation Accuracy: 0.6980
Epoch 30/300, Test Loss: 0.4923, Test Accuracy: 0.7867
Epoch 40/300, Training Loss: 0.4645, Validation Loss: 0.5525, Training Accuracy: 0.7805, Validation Accuracy: 0.6913
Epoch 40/300, Test Loss: 0.4913, Test Accuracy: 0.7800
Epoch 50/300, Training Loss: 0.4773, Validation Loss: 0.5514, Training Accuracy: 0.7805, Validation Accuracy: 0.6846
Epoch 50/300, Test Loss: 0.4884, Test Accuracy: 0.7867
Epoch 60/300, Training Loss: 0.4615, Validation Loss: 0.5461, Training Accuracy: 0.7747, Validation Accuracy: 0.7315
Epoch 60/300, Test Loss

#### Gradient Accumulation

In [16]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchsize_lr(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning = 0.007
optimizer = optim.Adam(model.parameters(), lr=learning)
num_epochs = 300
batch_size = 64
accumulation_steps = 4  # Number of batches over which gradients will be accumulated
best_val_loss = float('inf')

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    optimizer.zero_grad()  # Reset the gradients before accumulating
    
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        
        # Scale loss for accumulation steps
        loss = loss / accumulation_steps
        loss.backward()  # Accumulate the gradients
        
        # Accumulate gradients and perform optimizer step every 'accumulation_steps' batches
        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()  # Update the weights
            optimizer.zero_grad()  # Reset the gradients for the next set of accumulation steps
        
        running_loss += loss.item() * inputs.size(0) * accumulation_steps  # Adjust the loss accumulation
        predictions = torch.round(torch.sigmoid(outputs))
        correct_train += (predictions.squeeze(1) == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)
            val_predictions = torch.round(torch.sigmoid(val_outputs))
            correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
              f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        
# Evaluation on the test data
model.eval()
y_true = []
y_pred = []
test_loss = 0.0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
        test_loss += loss.item() * test_inputs.size(0)
        predictions = torch.round(torch.sigmoid(test_outputs))
        y_true.extend(test_labels.tolist())
        y_pred.extend(predictions.squeeze(1).tolist())

test_loss /= len(test_loader.dataset)
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

test_losses = [test_loss] * num_epochs
test_accuracies = [test_accuracy] * num_epochs

y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Learning Rate: {learning}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Epoch 10/300, Training Loss: 0.6980, Validation Loss: 0.6880, Training Accuracy: 0.5194, Validation Accuracy: 0.6040
Epoch 20/300, Training Loss: 0.6808, Validation Loss: 0.6791, Training Accuracy: 0.5782, Validation Accuracy: 0.6040
Epoch 30/300, Training Loss: 0.6721, Validation Loss: 0.6756, Training Accuracy: 0.5854, Validation Accuracy: 0.6644
Epoch 40/300, Training Loss: 0.6745, Validation Loss: 0.6546, Training Accuracy: 0.5940, Validation Accuracy: 0.6779
Epoch 50/300, Training Loss: 0.6538, Validation Loss: 0.6511, Training Accuracy: 0.6026, Validation Accuracy: 0.6846
Epoch 60/300, Training Loss: 0.6390, Validation Loss: 0.6440, Training Accuracy: 0.6370, Validation Accuracy: 0.6711
Epoch 70/300, Training Loss: 0.6602, Validation Loss: 0.6310, Training Accuracy: 0.6370, Validation Accuracy: 0.6980
Epoch 80/300, Training Loss: 0.6424, Validation Loss: 0.6250, Training Accuracy: 0.6485, Validation Accuracy: 0.6577
Epoch 90/300, Training Loss: 0.6307, Validation Loss: 0.6316, Tr

#### Best Model

In [32]:
class NN_batchnorm(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN_batchnorm, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Add BatchNorm
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(self.bn1(x))  # Apply BatchNorm
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(self.bn2(x))  # Apply BatchNorm
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(self.bn3(x))  # Apply BatchNorm
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Replace the model with the one using batch normalization
model = NN_batchnorm(input_size, hidden_size, output_size)


In [ ]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = NN_batchnorm(input_size, hidden_size, output_size)

criterion = nn.BCEWithLogitsLoss()

learning = 0.007
optimizer = optim.Adam(model.parameters(), lr=learning)
num_epochs = 300
batch_size = 64
best_val_loss = float('inf')

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, Y_test_tensor), batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
test_losses = []
train_accuracies = []
val_accuracies = []
test_accuracies_improved = []
start_time_improved = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predictions = torch.round(torch.sigmoid(outputs))
        correct_train += (predictions.squeeze(1) == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    

    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels.unsqueeze(1).float()).item() * val_inputs.size(0)

            val_predictions = torch.round(torch.sigmoid(val_outputs))
            correct_val += (val_predictions.squeeze(1) == val_labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    if (epoch+1)%10==0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
            f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    # Save the model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pkl')
    
    model.eval()
    y_true = []
    y_pred = []
    test_loss = 0.0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = model(test_inputs)
            loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
            test_loss += loss.item() * test_inputs.size(0)
            predictions = torch.round(torch.sigmoid(test_outputs))
            y_true.extend(test_labels.tolist())
            y_pred.extend(predictions.squeeze(1).tolist())

    test_loss /= len(test_loader.dataset)
    test_losses = test_losses.append(test_loss)
    test_accuracy = accuracy_score(y_true, y_pred)
    test_accuracies_improved.append(test_accuracy)  # Store test accuracy for this epoch

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

end_time_improved = time.time()
print(f"Training completed in: {(end_time_improved - start_time_improved) / 60:.2f} minutes")
model.load_state_dict(torch.load('best_model.pkl'))
# Evaluation on the test data
model.eval()
y_true = []
y_pred = []
test_loss = 0.0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        loss = criterion(test_outputs, test_labels.unsqueeze(1).float())
        test_loss += loss.item() * test_inputs.size(0)
        predictions = torch.round(torch.sigmoid(test_outputs))
        y_true.extend(test_labels.tolist())
        y_pred.extend(predictions.squeeze(1).tolist())

test_loss /= len(test_loader.dataset)
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Learning Rate: {learning}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

In [ ]:
# Example test accuracy data for the two models (you should replace these with your actual values)
# Assuming the accuracies are stored in test_accuracies_base and test_accuracies_improved lists
epochs_base = len(test_accuracies_base_model)  # Number of epochs run by the base model
epochs_improved = len(test_accuracies_improved)  # Number of epochs run by the improved model

# Assuming test_accuracies are for the first (base) model and test_accuracies_improved for the second (improved) model
epoch_range_base = np.arange(1, epochs_base + 1)
epoch_range_improved = np.arange(1, epochs_improved + 1)

# Create the plot
plt.figure(figsize=(10, 6))

# Plot test accuracy for the base model
plt.plot(epoch_range_base, test_accuracies_base_model, label='Base Model Test Accuracy', color='b', linestyle='--', marker='o')

# Plot test accuracy for the improved model
plt.plot(epoch_range_improved, test_accuracies_improved, label='Improved Model Test Accuracy', color='g', linestyle='-', marker='x')

# Add title and labels
plt.title('Test Accuracy Comparison: Base Model vs Improved Model')
plt.xlabel('Epochs')
plt.ylabel('Test Accuracy')

# Add legend
plt.legend()

# Show grid
plt.grid(True)

# Display the plot
plt.show()
print()

In [ ]:
# Plot training, validation, and test accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(num_epochs), train_accuracies, label='Training Accuracy')
plt.plot(range(num_epochs), val_accuracies, label='Validation Accuracy')
plt.plot(range(num_epochs), test_accuracies_improved, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training, Validation, and Test Accuracy')
plt.legend()
plt.show()

# Plot training, validation, and test loss
plt.figure(figsize=(10, 6))
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.plot(range(num_epochs), val_losses, label='Validation Loss')
plt.plot(range(num_epochs), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.show()


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# ROC curve
y_true_tensor = torch.tensor(y_true)
y_pred_tensor = torch.tensor(y_pred).float()

fpr, tpr, thresholds = roc_curve(y_true_tensor, y_pred_tensor)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()